In [6]:
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import pickle
import warnings
import mlflow
import mlflow.tensorflow
import mlflow.keras
from tqdm import tqdm
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download
from gensim.models import KeyedVectors
from tensorflow.config import list_physical_devices
from utils import split_data, filter_dataset
import dl
import time

In [2]:
SEED = 314
# Define the URI of the MLflow server and the name of the experiment
URI = "http://localhost:5000"
PATH_PARQUET = "../data/processed/df_preprocessed.parquet"
PATH_COLS = "../data/processed/columns.pkl"

In [3]:
# Remove FutureWarning alerts
warnings.filterwarnings("ignore", category=FutureWarning)

# Initialiser tqdm pour pandas
tqdm.pandas()

# Set a random seed
SEED = 314
np.random.seed(SEED)
print("Random seed set to", SEED)

# Check if GPU and CUDA are available
gpu = list_physical_devices("GPU")
print("Tensorflow framework: GPU is", "available" if gpu else "NOT AVAILABLE")

Random seed set to 314
Tensorflow framework: GPU is available


# **Séparation des données**

In [ ]:
# Load the pickle file containing the columns
with open(PATH_COLS, "rb") as f:
    cols = pickle.load(f)

# Load the parquet file
df = pd.read_parquet(
    PATH_PARQUET,
    engine="pyarrow",
    use_nullable_dtypes=False,
)

# Define the parameters for the split
proportion = 0.015  # approximatively 1.5% of 1.6 million : +/- 24000 rows
sampling = True
test_split = 0.2

# Split the data with the same SEED fixed
X_train_full, X_test_full, y_train, y_test = split_data(
    df,
    test_split=test_split,
    sampling=sampling,
    proportion=proportion,
)

In [ ]:
# Define the URI of the MLflow server and the name of the experiment
experiment = "neural_network_experiment"
cols_tracked = cols[3:]

# Set the tracking URI
mlflow.set_tracking_uri(URI)
# try to connect to the server
try:
    mlflow.tracking.get_tracking_uri()
except Exception as e:
    print(f"Cannot connect to the server : {URI}. Check the server status.")
    raise e
# Set, and create if necessary, the experiment
try:
    mlflow.create_experiment(experiment)
except Exception:
    pass
finally:
    mlflow.set_experiment(experiment)